In [1]:
from models import build_bayesian_linear_regression_model
from train import get_prepared_agreement_score_dataset
%load_ext autoreload
%autoreload 2

from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import pandas as pd
import arviz as az
import pymc3 as pm
from consts import *
from fastprogress.fastprogress import force_console_behavior
master_bar, progress_bar = force_console_behavior()

ERROR (theano.gpuarray): pygpu was configured but could not be imported or is too old (version 0.7 or higher required)
NoneType: None


In [2]:
def make_model_idata(model):
    with model:
        idata = pm.sample(draws=4000, tune=4000, cores=4, target_accept=0.99, init='adapt_diag', return_inferencedata=True)
    return idata
    
def logit_transform(data):
    return np.log(data / (1 - data))

def inverse_logit_transform(data):
    return 1 / (np.exp(-data) + 1)
    

In [3]:
# data_loc = '/home/mclancy/truespikes/hierarchical_linear_regression_data.hd5'
#
# data_store = pd.HDFStore(data_loc, mode='r')
# X_train = data_store['X_train']
# y_train = data_store['y_train']
# X_test = data_store['X_test']
# y_test = data_store['y_test']

X_train, y_train, X_test, y_test = get_prepared_agreement_score_dataset()

In [4]:
semipooled_model = build_bayesian_linear_regression_model(
X_train, y_train, metric_names=METRIC_NAMES,
    sorter_names=list(SORTER_NAMES.keys()),
    batch_size=None
)

varying_weights_and_bias_model = build_bayesian_linear_regression_model(
X_train, y_train, metric_names=METRIC_NAMES,
    sorter_names=list(SORTER_NAMES.keys()),
    batch_size=None
)

X_obs __str__ = [7780   13]
Weights __str__ = [[0.96781373 3.905024   5.4138613  ... 3.5512042  4.8546715  1.7091982 ]
 [0.96781373 3.905024   5.4138613  ... 3.5512042  4.8546715  1.7091982 ]
 [2.140692   5.0504885  1.5091169  ... 2.1828034  2.5767944  4.12758   ]
 ...
 [0.92268205 4.4211545  4.572377   ... 3.896056   4.4197907  1.7450457 ]
 [0.92268205 4.4211545  4.572377   ... 3.896056   4.4197907  1.7450457 ]
 [5.492206   3.9271328  3.0139067  ... 3.2939944  1.5145316  3.9632764 ]]
Bias __str__ = [7780   13]
X_obs __str__ = [7780   13]
Weights __str__ = [[3.7150297  3.2805972  3.0167544  ... 0.51656854 3.5611787  3.2609615 ]
 [3.7150297  3.2805972  3.0167544  ... 0.51656854 3.5611787  3.2609615 ]
 [4.211234   3.1548336  3.8003075  ... 4.055042   5.132705   5.2559857 ]
 ...
 [1.3650038  2.3135896  4.7583623  ... 4.185471   2.2177787  4.4475937 ]
 [1.3650038  2.3135896  4.7583623  ... 4.185471   2.2177787  4.4475937 ]
 [2.6378317  3.6254234  2.8381863  ... 3.5981982  3.3837285  3.4569

In [5]:
# semipooled_idata = make_model_idata(semipooled_model)
# semipooled_idata.to_netcdf('/home/mclancy/truespikes/traces/hierarchical_linear_regression/semipooled_trace.netcdf')
semipooled_idata = az.from_netcdf('/home/mclancy/truespikes/traces/hierarchical_linear_regression/semipooled_trace.netcdf')

#varying_weights_and_bias_idata = make_model_idata(varying_weights_and_bias_model)
#varying_weights_and_bias_idata.to_netcdf('/home/mclancy/truespikes/traces/hierarchical_linear_regression/varying_weights_and_bias_trace.netcdf')
varying_weights_and_bias_idata = az.from_netcdf('/home/mclancy/truespikes/traces/hierarchical_linear_regression/varying_weights_and_bias_trace.netcdf')

In [6]:
semipooled_summary = az.summary(varying_weights_and_bias_idata)


In [ ]:
semipooled_summary

In [8]:
varying_weights_and_bias_idata = az.summary(semipooled_idata)

KeyboardInterrupt: 

In [ ]:
varying_weights_and_bias_idata

In [ ]:
# az.plot_trace(semipooled_idata)
# plt.savefig('/home/mclancy/truespikes/figures/model_selection/bayesian_linear_regression/semipooled_traces.pdf')

az.plot_trace(
    varying_weights_and_bias_idata.posterior['weights'][4, :, :1000],
    var_names=["~z", "~chol"],
    lines=[("chol_corr", {}, 0.0)],
    compact=True,
    chain_prop={"ls": "-"},
)
plt.savefig('/home/mclancy/truespikes/figures/model_selection/bayesian_linear_regression/varying_weights_and_bias_traces.png')

In [ ]:
#Checking convergence in multiple dimensions is possible by checking similarity between marginal energy and proposed energy by sampler
az.plot_energy(varying_weights_and_bias_idata)
# az.plot_energy(semipooled_idata)

In [ ]:

y_preds = [sample_pred.mean() for sample_pred in (1 / (np.exp(-ppc['y']) + 1)).T]


zeros = np.ones(shape=y_test.shape[0])
baseline_rmse = np.sqrt(mean_squared_error((1 / (np.exp(-y_test) + 1)), zeros))

rmse = np.sqrt(mean_squared_error(inverse_logit_transform(y_test), inverse_logit_transform(y_preds)))

print(baseline_rmse)
print(rmse)